In [ ]:
from netsuitesdk2 import NetSuiteConnection
import json

In [ ]:
#Import Secrets

get_secrets = '{"NS_CONSUMER_KEY": "7f7442e79f25ffbd513166ffdd2c5c5db8077c73fbadd96cb3b3ed54643d420d", "NS_CONSUMER_SECRET": "a00604560403b093cc79fd88ffce2c9032fdc10f74f036c045298b383aa4a546", "NS_TOKEN_ID": "c521d3cb754d13189d9bf661a00729d5276c639f7101313a1c4b85e81682409b", "NS_TOKEN_SECRET": "3926130a8688b4512160676c36e5a33d64b92731302b484ba7c7cf8dcec16c96", "NETSUITE_ACCOUNT_ID": "4828879", "TEST_CONSUMER_KEY": "8b26283535c09ac458b294e405c6522be1679ad162128b0a8715515de086ac2c", "TEST_CONSUMER_SECRET": "ee74126313f614307c29516c877aef8dcc71660ce632158e9554a94aba3a1b9d", "TEST_TOKEN_ID": "003500b18b9714930c22dc7181469b02c1eebee8d593e26c1aada5ee4741aabb", "TEST_TOKEN_SECRET": "f378999dfc0506bfe1403e2e4443be4ba3322feb122e79981a23292859431675", "TEST_ACCOUNT_ID": "4828879_SB1"}'


secrets = json.loads(get_secrets)

# Production
NS_CONSUMER_KEY = secrets['NS_CONSUMER_KEY']
NS_CONSUMER_SECRET = secrets['NS_CONSUMER_SECRET']
NS_TOKEN_ID = secrets['NS_TOKEN_ID']
NS_TOKEN_SECRET = secrets['NS_TOKEN_SECRET']
NETSUITE_ACCOUNT_ID = secrets['NETSUITE_ACCOUNT_ID']

# Sandbox
TEST_CONSUMER_KEY = secrets['TEST_CONSUMER_KEY']
TEST_CONSUMER_SECRET = secrets['TEST_CONSUMER_SECRET']
TEST_TOKEN_ID = secrets['TEST_TOKEN_ID']
TEST_TOKEN_SECRET = secrets['TEST_TOKEN_SECRET']
TEST_ACCOUNT_ID = secrets['TEST_ACCOUNT_ID']


def connect(modes: str):
    if modes.lower() == 'test':

        nc = NetSuiteConnection(
        account=TEST_ACCOUNT_ID,
        consumer_key=TEST_CONSUMER_KEY,
        consumer_secret=TEST_CONSUMER_SECRET,
        token_key=TEST_TOKEN_ID,
        token_secret=TEST_TOKEN_SECRET,
        caching=False
        )
        try:
            return nc
        except:
            return "Connection Failed :("

    elif modes.lower() == 'production':

        nc = NetSuiteConnection(
            account=NETSUITE_ACCOUNT_ID,
            consumer_key=NS_CONSUMER_KEY,
            consumer_secret=NS_CONSUMER_SECRET,
            token_key=NS_TOKEN_ID,
            token_secret=NS_TOKEN_SECRET,
            caching=False
        )
        
        try:
            return nc

        except:
            return "Connection Failed :("

    else:
        return "Invalid modes selection! Choose either 'production' or 'test'."

    return "Something went wrong. Try again."

In [145]:
connection = connect('PRODUCTION')

In [146]:
connection.client.set_search_preferences(page_size=1000)

In [147]:
itemsearchrecord = connection.client.search_factory_adv('Item')

In [148]:
itemsearchrecord.savedSearchId = '1509'

In [149]:
itemsearchrecord.savedSearchScriptId='customsearch_gumps_ccc_item_export_3'

In [150]:
itemsearchrecord

{
    'criteria': None,
    'columns': None,
    'savedSearchId': '1509',
    'savedSearchScriptId': 'customsearch_gumps_ccc_item_export_3'
}

In [151]:
result = connection.client.search(itemsearchrecord)

In [152]:
result.searchId

'WEBSERVICES_4828879_08262022640937831316712855_1cadee888309f'

In [142]:
result.searchRowList.searchRow = []

In [143]:
result.searchRowList.searchRow

[]

In [ ]:
result.totalPages

In [ ]:
connection.client.searchMoreWithId('WEBSERVICES_4828879_082520227264760231986855303_4f2238dc28a1', 17)


In [ ]:
itemsearchrow = result.searchRowList.searchRow[0]['basic']

In [ ]:
for object_key in list(itemsearchrow.__values__.keys()):
    if itemsearchrow[object_key] != []:
        
        if object_key == 'internalId':
            print(object_key)
            print(itemsearchrow[object_key][0]['searchValue']['internalId'])
        elif object_key == 'customFieldList':
            pass
        else:
            print(object_key)
            print(itemsearchrow[object_key][0]['searchValue'])

In [144]:
import csv 
def get_headers(result):
    itemsearchrow = result.searchRowList.searchRow[0]['basic']
    headers = []
    for object_key in list(itemsearchrow.__values__.keys()):
        if itemsearchrow[object_key] != []:
            headers.append(object_key)
    return headers 
    
def extract_data(result, header):
    data = []
    search_id = result.searchId
    total_page = result.totalPages
    record_size = result.totalRecords
    #pagination
    for i in progressbar(range(1, total_page+1), "Computing: ", 40):

        page_data = connection.client.searchMoreWithId(search_id, i)
        page_size = len(page_data.searchRowList.searchRow)
        for page_idx in range(page_size):
            row = []
            itemsearchrow = page_data.searchRowList.searchRow[page_idx]['basic']
            for object_key in header:
                if itemsearchrow[object_key] != []:
                
                    if object_key == 'internalId':

                        row.append(itemsearchrow[object_key][0]['searchValue']['internalId'])
                    elif object_key == 'customFieldList':
                        row.append('')
                    else:
                        row.append(itemsearchrow[object_key][0]['searchValue'])
                else:
                    row.append('')
            data.append(row)
    return data
    
def save_result(conn, save_search_id:str, save_search_script_id:str):
    """_summary_

    Args:
        conn (_type_): _description_
        save_search_id (str): _description_
        save_search_script_id (str): _description_

    Returns:
        _type_: _description_
    """
    # generate save search record
    itemsearchrecord = conn.client.search_factory_adv('Item')
    itemsearchrecord.savedSearchId = save_search_id
    itemsearchrecord.savedSearchScriptId = save_search_script_id

    # query first pages of results
    result = connection.client.search(itemsearchrecord)
    search_id = result.searchId
    total_page = result.totalPages

    # query non pages
    if total_page > 1:
        for i in range(1, total_page+1):
            page_data = connection.client.searchMoreWithId(search_id, i)
        result.searchRowList.searchRow = result.searchRowList.searchRow + page_data.searchRowList.searchRow
    
    return result

def return_csv(header, data, name='test.csv'):

    with open(name, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(data)
    return f'done check {name}'

In [ ]:
header = get_headers(result)
data = extract_data(result, header)

In [ ]:
return_csv(header, data)

In [ ]:
len(data)

In [ ]:
for i in range(1, result.totalPages+1):
    page_data = connection.client.searchMoreWithId('WEBSERVICES_4828879_082520227264760231986855303_4f2238dc28a1', i)
    row = []
    itemsearchrow = page_data.searchRowList.searchRow[i]['basic']
    for object_key in header:
        if itemsearchrow[object_key] != []:
            
            if object_key == 'internalId':

                row.append(itemsearchrow[object_key][0]['searchValue']['internalId'])
            elif object_key == 'customFieldList':
                row.append('')
            else:
                row.append(itemsearchrow[object_key][0]['searchValue'])
    data.append(row)

In [ ]:
itemsearchrow[object_key]